In [1]:
import os
from bs4 import BeautifulSoup

In [2]:
slozka = "filmy"

In [3]:
def seznam_souboru(slozka):
    seznam = []
    for file in os.listdir(slozka):
        seznam.append(file)
    return(seznam)

In [4]:
def id_persony(url):
    if "cs/person/" in url:
        id_persony = url.split("/")[-2]
    else:
        id_persony = 0
    return(id_persony)

In [5]:
def persona(odkaz):
    id_persony = f"""{odkaz.text.strip()} ({odkaz["href"].split("/")[-2]})"""
    return(id_persony)

In [6]:
import pandas as pd

In [7]:
def scrape_pisne(slozka, soubor):
    
    with open(os.path.join(slozka, soubor), "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")
            
    pisne = pd.DataFrame()
                
    film = f"""{soup.find("h1").text.strip()} ({soup.find('meta', {'property': 'og:url'})['content'].split("/")[-2]})"""
    
    credit = soup.find(class_="film-credits-songs")

    if credit == None:
        
        pass
    
    else:
        
        pisnicky = credit.find_all('div', {'class': 'credit-item'})

        for p in pisnicky:
            
            pisen = []

            nazev = p.find("h3").text.strip()
            
            spany = p.find_all("span")
            
            hudba_k_pisni = []
            text_pisne = []
            zpiva = []

            kontrola = 0

            for s in spany:

                odkazy = s.find_all("a")

                if "ba k písni" in s.text.lower():

                    kontrola = 1
                    
                    pass

                if "xt pís" in s.text.lower():

                    kontrola = 2
                    
                    pass

                if s.text.strip().lower() == "zpívá":

                    kontrola = 3
                    
                    pass

                if kontrola == 1:

                    for o in odkazy:

                        if persona(o) not in hudba_k_pisni:

                            hudba_k_pisni.append(persona(o))
                            
                    pass

                if kontrola == 2:  

                    for o in odkazy:

                        if persona(o) not in text_pisne:

                            text_pisne.append(persona(o))
                            
                    pass

                if kontrola == 3:  

                    for o in odkazy:

                        if persona(o) not in zpiva:

                            zpiva.append(persona(o))
                            
                    pass
                
            if hudba_k_pisni == []:

                hudba_k_pisni = ""

            if text_pisne == []:

                text_pisne = ""

            if zpiva == []:

                zpiva = ""
                
            pisen = {"Film": film, "Píseň": nazev, "Hudba k písni": ", ".join(hudba_k_pisni), "Text písně": ", ".join(text_pisne), "Zpívá": ", ".join(zpiva)}
            
            pisen = pd.DataFrame(pisen, index=[0])
        
            pisne = pd.concat([pisne, pisen]).reset_index(drop = True)
                                
    return(pisne)

In [8]:
test = scrape_pisne(slozka, "396600-kdyby-tisic-klarinetu.html")
test

,Film,Píseň,Hudba k písni,Text písně,Zpívá
0,Kdyby tisíc klarinetů (396600),Kam utíkáš starý šneku,,,"Ivana Fišerová (31405), Rudolf Cortés (15020),..."
1,Kdyby tisíc klarinetů (396600),Babetta,Jiří Šlitr (2822),Jiří Suchý (2270),"Jiří Šlitr (2822), Jiří Suchý (2270)"
2,Kdyby tisíc klarinetů (396600),Tereza,Jiří Šlitr (2822),Jiří Suchý (2270),Waldemar Matuška (1787)
3,Kdyby tisíc klarinetů (396600),Píseň o rose,Jiří Šlitr (2822),Jiří Suchý (2270),Jiří Suchý (2270)
4,Kdyby tisíc klarinetů (396600),Spím,Jiří Šlitr (2822),Jiří Suchý (2270),"Karel Gott (3861), Jiří Suchý (2270), Jiří Šli..."
5,Kdyby tisíc klarinetů (396600),V opeře,Jiří Šlitr (2822),Jiří Suchý (2270),Hana Hegerová (13526)
6,Kdyby tisíc klarinetů (396600),Kde je ten míč,Jiří Šlitr (2822),Jiří Suchý (2270),"Jiří Suchý (2270), Jiří Šlitr (2822)"
7,Kdyby tisíc klarinetů (396600),Hallo Satchmo,Jiří Šlitr (2822),Jiří Suchý (2270),Jiří Jelínek (27512)
8,Kdyby tisíc klarinetů (396600),Tak abyste to věděla,Jiří Šlitr (2822),Jiří Suchý (2270),"Hana Hegerová (13526), Waldemar Matuška (1787)..."
9,Kdyby tisíc klarinetů (396600),Choď po špičkách,Jiří Šlitr (2822),Jiří Suchý (2270),Hana Hegerová (13526)


In [9]:
test = scrape_pisne(slozka, "103354-obcansky-prukaz.html")
print(test)
for x in test:
    print(len(x))

                        Film                           Píseň  \
0   Občanský průkaz (103354)                Most of the Time   
1   Občanský průkaz (103354)                  Obut a oblečen   
2   Občanský průkaz (103354)                        Paranoid   
3   Občanský průkaz (103354)                  Ballroom Blitz   
4   Občanský průkaz (103354)                Dobré ráno blues   
5   Občanský průkaz (103354)                         Nemakej   
6   Občanský průkaz (103354)                      Píseň kozi   
7   Občanský průkaz (103354)                    Eliášův oheň   
8   Občanský průkaz (103354)                           Lásko   
9   Občanský průkaz (103354)                Sun Is So Bright   
10  Občanský průkaz (103354)            Krajina posedlá tmou   
11  Občanský průkaz (103354)                        Joplinka   
12  Občanský průkaz (103354)                  Kovbojův nářek   
13  Občanský průkaz (103354)  Kdyby tady byla taková panenka   

                                       

In [10]:
test = scrape_pisne(slozka, "118284-dvojka.html")
test

,Film,Píseň,Hudba k písni,Text písně,Zpívá
0,Dvojka (118284),Funny,"a. m. almela (66179), Magdaléna Šalamounová (2...",,
1,Dvojka (118284),Tribute to Underworld,"DJ Smut (66180), Mawd (66181)",,
2,Dvojka (118284),Neon,Jan P. Muchow (2302),,
3,Dvojka (118284),Not As It Seems,Filip Míšek (9111),,
4,Dvojka (118284),No Name č. 1,Mawd (66181),,
5,Dvojka (118284),No Name č. 2,Mawd (66181),,
6,Dvojka (118284),Speedy Monkey,Mawd (66181),,
7,Dvojka (118284),A Sunday Afternoon on the Island of la Grande ...,"a. m. almela (66179), Magdaléna Šalamounová (2...",,
8,Dvojka (118284),Malá mořská víla,Aneta Langerová (19748),,
9,Dvojka (118284),Nicotine Lovers,"Karzi (26509), Gregory Finn (14726)",,


In [11]:
test = scrape_pisne(slozka, "396995-basnik.html")
test

,Film,Píseň,Hudba k písni,Text písně,Zpívá
0,Básník (396995),Cestička k Mayrovce,František Cajthaml (126903),,
1,Básník (396995),Kávu si osladím,Ladislav Štaidl (8125),Jiří Štaidl (2827),Karel Gott (3861)
2,Básník (396995),Rozkvetlý den,Radim Drejsl (83412),,
3,Básník (396995),Poutník,Karel Svoboda (2860),Zdeněk Rytíř (16047),Karel Gott (3861)


In [12]:
kolik = None

In [13]:
def frejm(slozka):
    pisnicky_pisnove = pd.DataFrame()
    total = len(seznam_souboru(slozka)[:kolik])
    count = 0
    for s in seznam_souboru(slozka)[:kolik]:
        count = count + 1
        print(f"{count} / {total} / {s}")
        line = (scrape_pisne(slozka, s))
        pisnicky_pisnove = pd.concat([pisnicky_pisnove, line])
    pisnicky_pisnove = pisnicky_pisnove.reset_index(drop = True)
    return(pisnicky_pisnove)

In [14]:
pisne_ceskych_filmu = frejm("filmy")

1 / 4580 / 10144-tabor-padlych-zen.html
2 / 4580 / 10173-eastern-krvavy-hugo.html
3 / 4580 / 102282-o-jecminkovi.html
4 / 4580 / 102314-zimni-vila.html
5 / 4580 / 10242-narkoblues.html
6 / 4580 / 103341-jan-hus-mse-za-tri-mrtve-muze.html
7 / 4580 / 103354-obcansky-prukaz.html
8 / 4580 / 103358-7-dni-hrichu.html
9 / 4580 / 103360-vojta-lavicka-nahoru-a-dolu.html
10 / 4580 / 103369-kuky-se-vraci.html
11 / 4580 / 103372-habermannuv-mlyn.html
12 / 4580 / 103374-dobre-placena-prochazka.html
13 / 4580 / 103402-nebe-peklo.html
14 / 4580 / 103495-lidice.html
15 / 4580 / 103499-medvedi-ostrovy.html
16 / 4580 / 103509-dont-stop.html
17 / 4580 / 104213-az-na-veky.html
18 / 4580 / 10574-elfilm.html
19 / 4580 / 106663-dum.html
20 / 4580 / 106666-soukromy-vesmir.html
21 / 4580 / 106667-mamas-papas.html
22 / 4580 / 10781-karu-suda-medvedi-srdce.html
23 / 4580 / 10960-unos-domu.html
24 / 4580 / 1118-nachove-plachty.html
25 / 4580 / 11221-mechanika.html
26 / 4580 / 112328-jmenem-krale.html
27 / 4580 / 

205 / 4580 / 180001-vsechno-je-sracka.html
206 / 4580 / 180025-laska-v-hrobe.html
207 / 4580 / 180317-rekvalifikace.html
208 / 4580 / 180386-muzi-v-nadeji.html
209 / 4580 / 18042-ctyri-lasky.html
210 / 4580 / 18045-fimfarum-jana-wericha.html
211 / 4580 / 180492-bastardi-2.html
212 / 4580 / 180545-svata-ctverice.html
213 / 4580 / 18056-franta-nebojsa.html
214 / 4580 / 18058-splneny-sen.html
215 / 4580 / 180585-okresni-prebor-posledni-zapas-pepika-hnatka.html
216 / 4580 / 180618-probudim-se-vcera.html
217 / 4580 / 181233-ve-stinu-bohyne-matky-zeme.html
218 / 4580 / 181912-zneuzivany.html
219 / 4580 / 181928-skolni-vylet.html
220 / 4580 / 181956-co-chytnes-v-zite.html
221 / 4580 / 181960-osina.html
222 / 4580 / 181961-cesta.html
223 / 4580 / 181962-stoh.html
224 / 4580 / 18203-z-mesta-cesta.html
225 / 4580 / 182231-libas-jako-dabel.html
226 / 4580 / 18262-pupendo.html
227 / 4580 / 18264-jedna-ruka-netleska.html
228 / 4580 / 18265-utek-do-budina.html
229 / 4580 / 182826-laska-je-laska.html

413 / 4580 / 395110-smich-a-plac.html
414 / 4580 / 395111-filmy-jana-krizeneckeho.html
415 / 4580 / 395111-vystavni-parkar-a-lepic-plakatu.html
416 / 4580 / 395112-to-nejlepsi-cislo.html
417 / 4580 / 395113-jarni-sen-stareho-mladence.html
418 / 4580 / 395114-hubicka.html
419 / 4580 / 395115-ponrepovo-kouzelnictvi.html
420 / 4580 / 395116-rudi-na-krtinach.html
421 / 4580 / 395117-rudi-na-zaletech.html
422 / 4580 / 395118-rudi-se-zeni.html
423 / 4580 / 395119-rudi-sportsman.html
424 / 4580 / 395120-sokove.html
425 / 4580 / 395121-ctyri-rocni-doby.html
426 / 4580 / 395122-dama-s-barzojem.html
427 / 4580 / 395123-falesny-hrac.html
428 / 4580 / 395124-pro-penize.html
429 / 4580 / 395125-zahadny-zlocin.html
430 / 4580 / 395126-zive-modely.html
431 / 4580 / 395127-americky-souboj.html
432 / 4580 / 395128-cholera-v-praze.html
433 / 4580 / 395129-estrella.html
434 / 4580 / 395130-faust.html
435 / 4580 / 395131-idyla-ze-stare-prahy.html
436 / 4580 / 395132-konec-milovani.html
437 / 4580 / 395133

616 / 4580 / 395313-on-a-ona.html
617 / 4580 / 395314-zkouska.html
618 / 4580 / 395315-bud-pripraven.html
619 / 4580 / 395316-carovne-oci.html
620 / 4580 / 395317-hrabenka-filangieri.html
621 / 4580 / 395318-kdyz-srdce-promluvi.html
622 / 4580 / 395319-madame-golvery.html
623 / 4580 / 395320-muz-bez-srdce.html
624 / 4580 / 395321-palicova-dcera.html
625 / 4580 / 395322-pepanek-nezdara.html
626 / 4580 / 395323-pro-a-proti.html
627 / 4580 / 395324-problematicky-gentleman.html
628 / 4580 / 395325-rasputin.html
629 / 4580 / 395326-tu-ten-kamen.html
630 / 4580 / 395327-unos-bankere-fuxe.html
631 / 4580 / 395328-za-oponou-smrti.html
632 / 4580 / 395329-zahadny-pripad-galginuv.html
633 / 4580 / 395330-zavet-podivinova.html
634 / 4580 / 395331-bily-raj.html
635 / 4580 / 395332-chytte-ho.html
636 / 4580 / 395333-dama-z-baru.html
637 / 4580 / 395334-devce-z-hor.html
638 / 4580 / 395335-dvoji-zivot.html
639 / 4580 / 395336-hrichy-v-manzelstvi.html
640 / 4580 / 395337-jindra-hrabenka-ostrovinova.h

814 / 4580 / 395512-pradlenka-jeho-jasnosti.html
815 / 4580 / 395513-tajemstvi-lekarovo.html
816 / 4580 / 395514-utrpeni-sede-sestry.html
817 / 4580 / 395515-v-kropenate-lhote-hori.html
818 / 4580 / 395516-vendelinuv-ocistec-a-raj.html
819 / 4580 / 395517-vse-pro-lasku.html
820 / 4580 / 395518-za-rodnou-hroudu.html
821 / 4580 / 395519-rada-lekare.html
822 / 4580 / 395520-tonka-sibenice.html
823 / 4580 / 395521-film-se-sasou-rasilovem-a-karlem-faltysem.html
824 / 4580 / 395522-kdyz-struny-lkaji-nemecka-verze.html
825 / 4580 / 395523-on-a-jeho-sestra-nemecka-verze.html
826 / 4580 / 395524-afera-plukovnika-redla-nemecka-verze.html
827 / 4580 / 395525-c-a-k-polni-marsalek-francouzska-verze.html
828 / 4580 / 395526-zena-ktera-se-smeje.html
829 / 4580 / 395527-svet-bez-hranic.html
830 / 4580 / 395528-afera-plukovnika-redla.html
831 / 4580 / 395529-pana-bumlicka-novy-domov.html
832 / 4580 / 395530-pensionat-petersilia.html
833 / 4580 / 395531-to-neznate-hadimrsku-nemecka-verze.html
834 / 4580

...

3741 / 4580 / 401686-paty-vsesokolsky-slet-v-praze.html
3742 / 4580 / 401687-kocarova-doprava.html
3743 / 4580 / 401688-jizda-prahou-otevrenou-tramvaji.html
3744 / 4580 / 401689-jubilejni-hold-mestskych-sboru-prazskych.html
3745 / 4580 / 401690-jubilejni-vystava-obchodni-a-zivnostenske-komory-1908-v-praz.html
3746 / 4580 / 401691-kvetinove-kocary.html
3747 / 4580 / 401692-polni-mse-na-staromestskem-namesti.html
3748 / 4580 / 401693-pruvod-delegaci-v-tradicnich-krojich.html
3749 / 4580 / 401694-pruvod-sokolstva-v-prostejove.html
3750 / 4580 / 401695-prvni-den-jarnich-dostihu-prazskych.html
3751 / 4580 / 401696-prvni-sjezd-ceskych-streleckych-jednot-v-praze.html
3752 / 4580 / 401697-reklamni-pruvod-ceskych-podniku-a-spolecnosti.html
3753 / 4580 / 401698-slavnostni-a-vsedni-vyjevy-z-vystaviste.html
3754 / 4580 / 401699-slavnost-otevreni-noveho-cechova-mostu.html
3755 / 4580 / 401700-slet-sokolstva-v-prostejove.html
3756 / 4580 / 401701-zahajeni-vystavy-za-ucasti-prazskeho-starosty.html
37

3922 / 4580 / 401872-hudebni-jaro.html
3923 / 4580 / 401873-chemie-v-kuchyni.html
3924 / 4580 / 401874-jak-se-dela-film.html
3925 / 4580 / 401875-junaci-v-prirode.html
3926 / 4580 / 401876-kapky-a-bubliny.html
3927 / 4580 / 401877-ladovi-furianti.html
3928 / 4580 / 401878-lide-a-parky.html
3929 / 4580 / 401879-miliony-na-dlazbach.html
3930 / 4580 / 401880-narodni-umelec-josef-lada.html
3931 / 4580 / 401881-nevime-dne.html
3932 / 4580 / 401882-obraz.html
3933 / 4580 / 401883-sklenena-suita.html
3934 / 4580 / 401884-to-jsou-bratri-v-triku.html
3935 / 4580 / 401885-cena-za-stesti.html
3936 / 4580 / 401886-letni-hokej.html
3937 / 4580 / 401887-podkarpatska-rus.html
3938 / 4580 / 401888-posadka-na-stite.html
3939 / 4580 / 401889-pred-startem-do-vesmiru.html
3940 / 4580 / 401890-ptak-ktery-kona-vojenskou-sluzbu.html
3941 / 4580 / 401891-reka-zivota-a-smrti.html
3942 / 4580 / 401892-uhori-maji-nabito.html
3943 / 4580 / 401893-na-strese.html
3944 / 4580 / 401896-skleneny-chleb.html
3945 / 4580

4110 / 4580 / 402068-3bobule.html
4111 / 4580 / 402069-havel.html
4112 / 4580 / 402070-kralove-videa.html
4113 / 4580 / 402071-kiruna-prekrasny-novy-svet.html
4114 / 4580 / 402072-milenec-leta.html
4115 / 4580 / 402073-cesta-do-nemozna.html
4116 / 4580 / 402075-svet-podle-muchy.html
4117 / 4580 / 402077-poslouchej.html
4118 / 4580 / 402078-zenska-pomsta.html
4119 / 4580 / 402079-modelar.html
4120 / 4580 / 402080-mlsne-medvedi-pribehy-v-kinech.html
4121 / 4580 / 402081-v-siti-18.html
4122 / 4580 / 402082-alchymicka-pec.html
4123 / 4580 / 402083-jan-jedlicka-stopy-krajiny.html
4124 / 4580 / 402084-frem.html
4125 / 4580 / 402085-nebe.html
4126 / 4580 / 402086-nova-sichta.html
4127 / 4580 / 402087-zmatek.html
4128 / 4580 / 402088-provincni-mestecko-e.html
4129 / 4580 / 402089-anny.html
4130 / 4580 / 402090-vysehrad-seryjal.html
4131 / 4580 / 402091-princezna-zakleta-v-case.html
4132 / 4580 / 402092-sarlatan.html
4133 / 4580 / 402093-les.html
4134 / 4580 / 402094-babovky.html
4135 / 4580 / 

4309 / 4580 / 45361-kvaska.html
4310 / 4580 / 45536-mir-s-tuleni.html
4311 / 4580 / 45546-ine-svety.html
4312 / 4580 / 45588-pasti-pasti-pasticky.html
4313 / 4580 / 45796-kozi-pribeh-povesti-stare-prahy.html
4314 / 4580 / 45816-a-bude-hur.html
4315 / 4580 / 46221-kauza-cervanova.html
4316 / 4580 / 46301-kupredu-leva-kupredu-prava.html
4317 / 4580 / 46441-maj.html
4318 / 4580 / 46868-pul-ctvrte.html
4319 / 4580 / 47114-casino-royale.html
4320 / 4580 / 47118-po-hlave-do-prdele.html
4321 / 4580 / 47395-snezenky-a-machri-po-25-letech.html
4322 / 4580 / 47585-cesky-fenomen-tanecni.html
4323 / 4580 / 48211-saxana-a-lexikon-kouzel.html
4324 / 4580 / 48348-pad.html
4325 / 4580 / 48378-oko-ve-zdi.html
4326 / 4580 / 48383-chytte-doktora.html
4327 / 4580 / 48718-marcela.html
4328 / 4580 / 48860-posledni-plavky.html
4329 / 4580 / 48950-vetrelci-v-coloradu.html
4330 / 4580 / 495-cabriolet.html
4331 / 4580 / 50479-jedne-noci-v-jednom-meste.html
4332 / 4580 / 50480-ulity.html
4333 / 4580 / 50482-jedn

4507 / 4580 / 8249-hrad-z-pisku.html
4508 / 4580 / 8254-saturnin.html
4509 / 4580 / 8259-strelec.html
4510 / 4580 / 82817-draci.html
4511 / 4580 / 8296-indianske-leto.html
4512 / 4580 / 8404-valka-barev.html
4513 / 4580 / 8405-divoke-pivo.html
4514 / 4580 / 8461-fany.html
4515 / 4580 / 8462-jak-chutna-smrt.html
4516 / 4580 / 8480-kone-na-betone.html
4517 / 4580 / 8531-kouzelny-mesec.html
4518 / 4580 / 8552-ani-smrt-nebere.html
4519 / 4580 / 856-kruh.html
4520 / 4580 / 8578-horke-leto.html
4521 / 4580 / 87180-jan-69.html
4522 / 4580 / 8749-uz.html
4523 / 4580 / 8785-buldocek.html
4524 / 4580 / 8786-posledni-stisko-pana-zorze.html
4525 / 4580 / 8787-garden-party-s-vernisazi.html
4526 / 4580 / 88837-pripad-neverne-klary.html
4527 / 4580 / 8898-v-zahrade.html
4528 / 4580 / 8961-mladi-muzi-poznavaji-svet.html
4529 / 4580 / 8966-holcicky-na-zivot-a-na-smrt.html
4530 / 4580 / 8967-zahrada.html
4531 / 4580 / 89876-the-resolution.html
4532 / 4580 / 8999-mnaga-happy-end.html
4533 / 4580 / 9007-c

In [15]:
pisne_ceskych_filmu

,Film,Píseň,Hudba k písni,Text písně,Zpívá
0,Občanský průkaz (103354),Most of the Time,Bob Dylan (48949),,Bob Dylan (48949)
1,Občanský průkaz (103354),Obut a oblečen,Petr Ostrouchov (5833),,"Matouš Vrba (76615), Petr Ostrouchov (5833)"
2,Občanský průkaz (103354),Paranoid,"Tony Iommi (5649), Bill Ward (76634), Ozzy Osb...","Tony Iommi (5649), Bill Ward (76634), Ozzy Osb...",Black Sabbath (76649)
3,Občanský průkaz (103354),Ballroom Blitz,"Michael Chapman (76637), Nicholas Chinn (76638)","Michael Chapman (76637), Nicholas Chinn (76638)",Sweet (76650)
4,Občanský průkaz (103354),Dobré ráno blues,,Vladimír Ptaszek (76643),"Ľuboš Beňa (76632), Matěj Ptaszek (30591)"
...,...,...,...,...,...
10257,Baader Meinhof Komplex (99894),Dizzy Miss Lizzy,Larry E. Williams (51312),Larry E. Williams (51312),
10258,Baader Meinhof Komplex (99894),Child in Time,"Jon Lord (51313), Ian Gillan (51314), Roger Gl...","Jon Lord (51313), Ian Gillan (51314), Roger Gl...",skupina Deep Purple (51328)
10259,Baader Meinhof Komplex (99894),"Please, Please, Please",Michael Gerdes-Heitling (51317),"Michael Gerdes-Heitling (51317), Jens-Peter Fu...",
10260,Baader Meinhof Komplex (99894),Blowin' in the Wind,Bob Dylan (48949),Bob Dylan (48949),Bob Dylan (48949)


In [16]:
def do_seznamu(polozka):
    try:
        if "," in polozka:
            polozka = str(polozka).split(",")
            polozka = [p.strip() for p in polozka]
    except:
        pass
    return(polozka)

In [17]:
def znuluj(x):
    if x == "":
        y = None
    else:
        y = x
    return(y)

In [18]:
pisne_ceskych_filmu["Hudba k písni"] = pisne_ceskych_filmu["Hudba k písni"].apply(lambda x: do_seznamu(x))
pisne_ceskych_filmu["Text písně"] = pisne_ceskych_filmu["Text písně"].apply(lambda x: do_seznamu(x))
pisne_ceskych_filmu["Zpívá"] = pisne_ceskych_filmu["Zpívá"].apply(lambda x: do_seznamu(x))

In [19]:
pisne_ceskych_filmu["Hudba k písni"] = pisne_ceskych_filmu["Hudba k písni"].apply(lambda x: znuluj(x))
pisne_ceskych_filmu["Text písně"] = pisne_ceskych_filmu["Text písně"].apply(lambda x: znuluj(x))
pisne_ceskych_filmu["Zpívá"] = pisne_ceskych_filmu["Zpívá"].apply(lambda x: znuluj(x))

In [20]:
import os

In [21]:
pisne_ceskych_filmu.to_json(os.path.join("data", "pisne.json"), orient="records")